In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Student Performance Prediction - End-to-End ML Pipeline
# -------------------------------------------------------
# Covers: Regression + Classification, Preprocessing, Model Comparison

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor, XGBClassifier

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# ==============================
# Load Dataset
# ==============================
# Kaggle dataset: https://www.kaggle.com/datasets/spscientist/students-performance-in-exams
df = pd.read_csv("/kaggle/input/students-performance-in-exams/StudentsPerformance.csv")

print("Shape:", df.shape)
print(df.head())

# ==============================
# Feature Engineering
# ==============================
# Convert categorical columns
categorical = df.select_dtypes(include=["object"]).columns.tolist()
numerical = df.select_dtypes(exclude=["object"]).columns.tolist()

# Example: average score as regression target
df["average_score"] = df[["math score", "reading score", "writing score"]].mean(axis=1)

# Example: classification target (pass/fail if avg >= 50)
df["passed"] = (df["average_score"] >= 50).astype(int)

# ==============================
# Preprocessing Pipelines
# ==============================
num_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

cat_transformer = Pipeline(steps=[
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, numerical),
        ("cat", cat_transformer, categorical)
    ]
)

# ==============================
# Regression Models
# ==============================
X = df.drop(columns=["average_score", "passed"])
y_reg = df["average_score"]

X_train, X_test, y_train, y_test = train_test_split(X, y_reg, test_size=0.2, random_state=42)

reg_models = {
    "Linear Regression": LinearRegression(),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=200, random_state=42),
    "XGBoost Regressor": XGBRegressor(n_estimators=300, learning_rate=0.05, random_state=42)
}

print("\n===== Regression Models =====")
for name, model in reg_models.items():
    reg_pipe = Pipeline(steps=[("preprocessor", preprocessor),
                               ("regressor", model)])
    reg_pipe.fit(X_train, y_train)
    y_pred = reg_pipe.predict(X_test)
    
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    
    print(f"{name}: RMSE={rmse:.4f}, R²={r2:.4f}")

# ==============================
# Classification Models
# ==============================
X = df.drop(columns=["average_score", "passed"])
y_clf = df["passed"]

X_train, X_test, y_train, y_test = train_test_split(X, y_clf, test_size=0.2, random_state=42, stratify=y_clf)

clf_models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest Classifier": RandomForestClassifier(n_estimators=200, random_state=42),
    "XGBoost Classifier": XGBClassifier(n_estimators=300, learning_rate=0.05, use_label_encoder=False, eval_metric="logloss", random_state=42)
}

print("\n===== Classification Models =====")
for name, model in clf_models.items():
    clf_pipe = Pipeline(steps=[("preprocessor", preprocessor),
                               ("classifier", model)])
    clf_pipe.fit(X_train, y_train)
    y_pred = clf_pipe.predict(X_test)
    y_prob = clf_pipe.predict_proba(X_test)[:, 1]
    
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_prob)
    
    print(f"{name}: Accuracy={acc:.4f}, Precision={prec:.4f}, Recall={rec:.4f}, F1={f1:.4f}, ROC-AUC={roc:.4f}")
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Shape: (1000, 8)
   gender race/ethnicity parental level of education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male        group A          associate's degree  free/reduced   
4    male        group C                some college      standard   

  test preparation course  math score  reading score  writing score  
0                    none          72             72             74  
1               completed          69             90             88  
2                    none          90             95             93  
3                    none          47             57             44  
4                    none          76             78             75  

===== Regression Models =====
Linear Regression: RMSE=0.0000, R²=1.0000
Random Forest Regressor: RMSE=1.1100, R²=0.9943
XGBoost Regressor: RM